## Loading Data

In [ ]:
import zipfile
with zipfile.ZipFile("/content/novozymes-enzyme-stability-prediction.zip", "r") as zip_ref:
  zip_ref.extractall("Data")

In [ ]:
import pandas as pd
train_df = pd.read_csv("/content/Data/train.csv")

In [ ]:
train_df.head()

In [ ]:
train_updated_df = pd.read_csv("/content/Data/train_updates_20220929.csv")
train_updated_df.head()

## Updated CSV

In [ ]:
import pandas as pd


df_train = pd.read_csv("/content/Data/train.csv", index_col="seq_id")
df_train_updates = pd.read_csv("/content/Data/train_updates_20220929.csv", index_col="seq_id")

all_features_nan = df_train_updates.isnull().all("columns")

drop_indices = df_train_updates[all_features_nan].index
df_train = df_train.drop(index=drop_indices)

swap_ph_tm_indices = df_train_updates[~all_features_nan].index
df_train.loc[swap_ph_tm_indices, ["pH", "tm"]] = df_train_updates.loc[swap_ph_tm_indices, ["pH", "tm"]]

In [ ]:
df_train_updates

In [ ]:
df_train

In [ ]:
df = pd.read_csv("/content/Data/train.csv")
updates = pd.read_csv("/content/Data/train_updates_20220929.csv")

drop_seq_ids = updates[updates["pH"].isnull()]["seq_id"].values
change_seq_ids = updates[updates["pH"].notnull()]["seq_id"].values

# drop
df.drop(df[df.seq_id.isin(drop_seq_ids)].index, inplace=True)

# change
df.loc[df.seq_id.isin(change_seq_ids), ["pH", "tm"]] = updates[
    updates.seq_id.isin(change_seq_ids)
][["pH", "tm"]].values

df.to_csv("updated_train.csv", index=False)

In [ ]:
df_updated = pd.read_csv("/content/updated_train.csv")

In [ ]:
df_updated.head()

## EDA

In [ ]:
conda install biopandas

In [ ]:
import biopandas

In [ ]:
somethin = ppdb.df["ATOM"]

In [ ]:
wt = somethin.groupby('residue_number').b_factor.agg('first').values